In [2]:
import pandas as pd

books = pd.read_csv('categorised_books.csv')

In [6]:
from transformers import pipeline
classifier = pipeline('text-classification', model='j-hartmann/emotion-english-distilroberta-base', top_k=None)

Device set to use mps:0


In [7]:
classifier('I love this')

[[{'label': 'joy', 'score': 0.9845667481422424},
  {'label': 'surprise', 'score': 0.004927210509777069},
  {'label': 'sadness', 'score': 0.004531432408839464},
  {'label': 'neutral', 'score': 0.0034753207582980394},
  {'label': 'anger', 'score': 0.0013875329168513417},
  {'label': 'disgust', 'score': 0.0007134057814255357},
  {'label': 'fear', 'score': 0.0003984913637395948}]]

In [8]:
books['description'][0]

'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gilead is a song of celebration and acceptance of the best and the worst the world ha

In [9]:
classifier(books['description'][0])

[[{'label': 'fear', 'score': 0.6548422574996948},
  {'label': 'neutral', 'score': 0.16985155642032623},
  {'label': 'sadness', 'score': 0.11640843003988266},
  {'label': 'surprise', 'score': 0.020700642839074135},
  {'label': 'disgust', 'score': 0.019100701436400414},
  {'label': 'joy', 'score': 0.015161211602389812},
  {'label': 'anger', 'score': 0.003935141488909721}]]

In [10]:
# Splitting each description down into its sentences to capture all emotions better
classifier(books['description'][0].split('.'))

[[{'label': 'surprise', 'score': 0.7296021580696106},
  {'label': 'neutral', 'score': 0.14038625359535217},
  {'label': 'fear', 'score': 0.06816215068101883},
  {'label': 'joy', 'score': 0.0479423962533474},
  {'label': 'anger', 'score': 0.009156353771686554},
  {'label': 'disgust', 'score': 0.0026284793857485056},
  {'label': 'sadness', 'score': 0.0021221640054136515}],
 [{'label': 'neutral', 'score': 0.44937190413475037},
  {'label': 'disgust', 'score': 0.2735902965068817},
  {'label': 'joy', 'score': 0.1090828999876976},
  {'label': 'sadness', 'score': 0.09362740814685822},
  {'label': 'anger', 'score': 0.04047819972038269},
  {'label': 'surprise', 'score': 0.02697022259235382},
  {'label': 'fear', 'score': 0.00687906239181757}],
 [{'label': 'neutral', 'score': 0.6462162137031555},
  {'label': 'sadness', 'score': 0.24273328483104706},
  {'label': 'disgust', 'score': 0.043422579765319824},
  {'label': 'surprise', 'score': 0.028300493955612183},
  {'label': 'joy', 'score': 0.014211435

In [12]:
sentences = books['description'][0].split('.')
predictions = classifier(sentences)

In [13]:
predictions[0]

[{'label': 'surprise', 'score': 0.7296021580696106},
 {'label': 'neutral', 'score': 0.14038625359535217},
 {'label': 'fear', 'score': 0.06816215068101883},
 {'label': 'joy', 'score': 0.0479423962533474},
 {'label': 'anger', 'score': 0.009156353771686554},
 {'label': 'disgust', 'score': 0.0026284793857485056},
 {'label': 'sadness', 'score': 0.0021221640054136515}]

In [14]:
sentences[3]

' Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist'

In [15]:
predictions[3]

[{'label': 'fear', 'score': 0.9281688332557678},
 {'label': 'anger', 'score': 0.03219037875533104},
 {'label': 'neutral', 'score': 0.012808619067072868},
 {'label': 'sadness', 'score': 0.00875683780759573},
 {'label': 'surprise', 'score': 0.008597852662205696},
 {'label': 'disgust', 'score': 0.008431759662926197},
 {'label': 'joy', 'score': 0.001045577460899949}]

In [16]:
sorted(predictions[0], key=lambda x: x['label'])

[{'label': 'anger', 'score': 0.009156353771686554},
 {'label': 'disgust', 'score': 0.0026284793857485056},
 {'label': 'fear', 'score': 0.06816215068101883},
 {'label': 'joy', 'score': 0.0479423962533474},
 {'label': 'neutral', 'score': 0.14038625359535217},
 {'label': 'sadness', 'score': 0.0021221640054136515},
 {'label': 'surprise', 'score': 0.7296021580696106}]

In [20]:
import numpy as np
emotion_labels = ['anger', 'disgust', 'fear', 'happiness', 'sadness', 'surprise']
isbn = []
emotion_scores = {label: [] for label in emotion_labels}

def calculate_emotion_score(preds):
    per_emotion_scores = {label: [] for  label in emotion_labels}
    for pred in preds:
        sorted_preds = sorted(pred, key=lambda x: x['label'])
        for index, label in enumerate(emotion_labels):
            per_emotion_scores[label].append(sorted_preds[index]['score'])

    return {label: np.max(scores) for label, scores in per_emotion_scores.items()}

In [21]:
for i in range(10):
    isbn.append(books['isbn13'][i])
    sentences = books['description'][i].split('.')
    predictions = classifier(sentences)
    max_scores = calculate_emotion_score(predictions)
    for label in emotion_labels:
        emotion_scores[label].append(max_scores[label])

In [22]:
emotion_scores

{'anger': [np.float64(0.06413352489471436),
  np.float64(0.6126186847686768),
  np.float64(0.06413352489471436),
  np.float64(0.35148337483406067),
  np.float64(0.08141209185123444),
  np.float64(0.23222483694553375),
  np.float64(0.5381850600242615),
  np.float64(0.06413352489471436),
  np.float64(0.30067047476768494),
  np.float64(0.06413352489471436)],
 'disgust': [np.float64(0.2735902965068817),
  np.float64(0.34828585386276245),
  np.float64(0.1040065661072731),
  np.float64(0.15072239935398102),
  np.float64(0.18449506163597107),
  np.float64(0.7271748185157776),
  np.float64(0.1558549702167511),
  np.float64(0.1040065661072731),
  np.float64(0.2794809639453888),
  np.float64(0.17792749404907227)],
 'fear': [np.float64(0.9281688332557678),
  np.float64(0.9425276517868042),
  np.float64(0.9723208546638489),
  np.float64(0.3607071340084076),
  np.float64(0.09504341334104538),
  np.float64(0.05136272683739662),
  np.float64(0.7474281787872314),
  np.float64(0.404495507478714),
  np.

In [23]:
from tqdm import tqdm

emotion_labels = ['anger', 'disgust', 'fear', 'happiness', 'sadness', 'surprise']
isbn = []
emotion_scores = {label: [] for label in emotion_labels}

for i in tqdm(range(len(books))):
    isbn.append(books['isbn13'][i])
    sentences = books['description'][i].split('.')
    predictions = classifier(sentences)
    max_scores = calculate_emotion_score(predictions)
    for label in emotion_labels:
        emotion_scores[label].append(max_scores[label])

100%|██████████| 5197/5197 [05:43<00:00, 15.14it/s]


In [25]:
emotions_df = pd.DataFrame(emotion_scores)
emotions_df['isbn13'] = isbn

In [26]:
emotions_df.head()

,anger,disgust,fear,happiness,sadness,surprise,isbn13
0,0.064134,0.273590,0.928169,0.932798,0.646216,0.967158,9780002005883
1,0.612619,0.348286,0.942528,0.704422,0.887940,0.111690,9780002261982
2,0.064134,0.104007,0.972321,0.767237,0.549477,0.111690,9780006178736
3,0.351483,0.150722,0.360707,0.251881,0.732687,0.111690,9780006280897
4,0.081412,0.184495,0.095043,0.040564,0.884389,0.475881,9780006280934


In [27]:
books = pd.merge(books, emotions_df, on='isbn13')

In [28]:
books

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,...,words_count,tagged_description,predicted_categories_x,predicted_categories_y,anger,disgust,fear,happiness,sadness,surprise
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,...,199,9780002005883 A NOVEL THAT READERS and critics...,NaN,NaN,0.064134,0.273590,0.928169,0.932798,0.646216,0.967158
1,9780002261982,0002261987,Spider's Web : A Novel,Charles Osborne;Agatha Christie,Fiction,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,...,205,9780002261982 A new 'Christie for Christmas' -...,Fiction,Fiction,0.612619,0.348286,0.942528,0.704422,0.887940,0.111690
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,...,57,"9780006178736 A memorable, mesmerizing heroine...",NaN,NaN,0.064134,0.104007,0.972321,0.767237,0.549477,0.111690
3,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Nonfiction,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,...,45,9780006280897 Lewis' work on the nature of lov...,Nonfiction,Nonfiction,0.351483,0.150722,0.360707,0.251881,0.732687,0.111690
4,9780006280934,0006280935,The Problem of Pain,Clive Staples Lewis,Nonfiction,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,...,75,"9780006280934 ""In The Problem of Pain, C.S. Le...",Nonfiction,Nonfiction,0.081412,0.184495,0.095043,0.040564,0.884389,0.475881
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5192,9788172235222,8172235224,Mistaken Identity,Nayantara Sahgal,Fiction,http://books.google.com/books/content?id=q-tKP...,On A Train Journey Home To North India After L...,2003.0,2.93,324.0,...,288,9788172235222 On A Train Journey Home To North...,Fiction,Fiction,0.148208,0.030643,0.919165,0.255170,0.853723,0.980877
5193,9788173031014,8173031010,Journey to the East,Hermann Hesse,Nonfiction,http://books.google.com/books/content?id=rq6JP...,This book tells the tale of a man who goes on ...,2002.0,3.70,175.0,...,63,9788173031014 This book tells the tale of a ma...,Nonfiction,Nonfiction,0.064134,0.114383,0.051363,0.400263,0.883198,0.111690
5194,9788179921623,817992162X,The Monk Who Sold His Ferrari: A Fable About F...,Robin Sharma,Fiction,http://books.google.com/books/content?id=c_7mf...,"Wisdom to Create a Life of Passion, Purpose, a...",2003.0,3.82,198.0,...,117,9788179921623 Wisdom to Create a Life of Passi...,Fiction,Fiction,0.009997,0.009929,0.339217,0.947779,0.375755,0.066685
5195,9788185300535,8185300534,I Am that : Talks with Sri Nisargadatta Maharaj,Sri Nisargadatta Maharaj;Sudhakar S. Dikshit,Nonfiction,http://books.google.com/books/content?id=Fv_JP...,This collection of the timeless teachings of o...,1999.0,4.51,531.0,...,174,9788185300535 This collection of the timeless ...,NaN,NaN,0.064134,0.104007,0.459270,0.759455,0.951104,0.368110


In [31]:
books.drop(['predicted_categories_x', 'predicted_categories_y'], axis=1, inplace=True)

In [32]:
books.head()

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,...,missing_description,age_of_book,words_count,tagged_description,anger,disgust,fear,happiness,sadness,surprise
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,...,0,21.0,199,9780002005883 A NOVEL THAT READERS and critics...,0.064134,0.273590,0.928169,0.932798,0.646216,0.967158
1,9780002261982,0002261987,Spider's Web : A Novel,Charles Osborne;Agatha Christie,Fiction,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,...,0,25.0,205,9780002261982 A new 'Christie for Christmas' -...,0.612619,0.348286,0.942528,0.704422,0.887940,0.111690
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,...,0,32.0,57,"9780006178736 A memorable, mesmerizing heroine...",0.064134,0.104007,0.972321,0.767237,0.549477,0.111690
3,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Nonfiction,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,...,0,23.0,45,9780006280897 Lewis' work on the nature of lov...,0.351483,0.150722,0.360707,0.251881,0.732687,0.111690
4,9780006280934,0006280935,The Problem of Pain,Clive Staples Lewis,Nonfiction,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,...,0,23.0,75,"9780006280934 ""In The Problem of Pain, C.S. Le...",0.081412,0.184495,0.095043,0.040564,0.884389,0.475881


In [33]:
books.to_csv('books_with_emotions.csv', index=False)